# 4 Modeling - Investigating San Francisco Housing Prices Through Police Incident Reports and 311 Cases<a id='Modeling'></a>

## 1 Contents<a id='1_Contents'></a>
* [Modeling - Investigating San Francisco Housing Prices Through Police Incident Reports and 311 Cases](#Modeling)
  * [1 Contents](#1_Contents)
  * [2 Introduction](#2_Introduction)
  * [3 Imports](#3_Imports)
  * [4 Load The Data](#4_Load_The_Data)
  * [5 Create Train and Test Splits](#5_Create_Train_and_Test_Splits)
  * [6 Models](#6_Models)
    * [6.1 Model 1 - Standard Scaler, PCA, Linear Regression](#6.1_Model_1_-_StandardScaler_PCA_LinearRegression)
    * [6.2 Model 2 - Robust Scaler, PCA, Linear Regression](#6.2_Model_2_-_RobustScaler_PCA_LinearRegression)
    * [6.3 Model 3 - Robust Scaler, SelectKBest, Linear Regression](#6.3_Model_3_-_RobustScaler_SelectKBest_LinearRegression)
    * [6.4 Model 4 - Robust Scaler, PCA, Random Forest](#6.4_Model_4_-_RobustScaler_PCA_RandomForest)
  * [7 Model Evaluation](#7_Model_Evaluation)

## 2 Introduction<a id='2_Introduction'></a>

In this notebook, we will build different machine learning models using hyperparameter tuning and compare them to determine the best model that most accurately predicts the relationship between San Francisco housing prices and police incident reports and 311 cases. 

We will use the file `Post_EDA_SF_Combined_SFPD_311_Housing.csv` that was created in our previous Jupyter Notebook, `2-Exploratory_Data_Analysis.ipynb`. This file contains all SF police incident reports, 311 cases, and housing sales data aggregated by month and by neighborhood, from January 2018 up to and including September 2020, wherein each row is an observation with a distinct pairing on month-year and each column represents a possible feature to be used in modelling.

## 3 Imports<a id='3_Imports'></a>

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import scale, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


## 4 Load The Data<a id='4_Load_The_Data'></a>

In [2]:
df = pd.read_csv('../data/Post_EDA_SF_Combined_SFPD_311_Housing.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188 entries, 0 to 1187
Data columns (total 82 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Year Month                                    1188 non-null   int64  
 1   Neighborhood                                  1188 non-null   object 
 2   Arson                                         1188 non-null   int64  
 3   Assault                                       1188 non-null   int64  
 4   Burglary                                      1188 non-null   int64  
 5   Case Closure                                  1188 non-null   int64  
 6   Civil Sidewalks                               1188 non-null   int64  
 7   Courtesy Report                               1188 non-null   int64  
 8   Disorderly Conduct                            1188 non-null   int64  
 9   Drug Offense                                  1188 non-null   i

## 5 Create Train and Test Splits<a id='5_Create_Train_and_Test_Splits'></a>

In our previous notebook `3-Preprocessing_and_Training.ipynb`, we manually created the train and test splits so as to avoid data leakage.

Here, we will split out 85% of the data into the training set, leaving 15% of the data in the test set.

In [4]:
# Look at how many months of data we have
print('Earliest date:',df['Year Month'].min(), 'Latest date:', df['Year Month'].max(), 'Number of months:', len(df['Year Month'].unique()))

Earliest date: 201801 Latest date: 202009 Number of months: 33


In [5]:
# We will split the data with 28 months in train and 5 months in test, approx 85/15 train/test split
train = df[df['Year Month'] < 202005]
test = df[df['Year Month'] >= 202005]

In [6]:
train.shape, test.shape

((1008, 82), (180, 82))

In [7]:
X_train = train.drop(columns='Median Sale Price')
X_test = test.drop(columns='Median Sale Price')
y_train = train['Median Sale Price']
y_test = test['Median Sale Price']

In [8]:
X_train.shape, X_test.shape

((1008, 81), (180, 81))

In [9]:
y_train.shape, y_test.shape

((1008,), (180,))

In [10]:
# Save the 'Year Month' and 'Neighborhood' columns from the train/test data into ids_train and ids_test
# and drop these from 'X_train' and 'X_test'
ids_list = ['Year Month', 'Neighborhood']
ids_train = X_train[ids_list]
ids_test = X_test[ids_list]
X_train.drop(columns=ids_list, inplace=True)
X_test.drop(columns=ids_list, inplace=True)
X_train.shape, X_test.shape

((1008, 79), (180, 79))

## 6 Models<a id='6_Models'></a>

We're ready to create and evaluate some models. In order to do this, for each model, we will create a pipeline, then pass it into `GridSearchCV` to determine our optimal parameters for that model.

Each pipeline will consist of:
  * scaler: since our features contain numbers that vary by orders of magniture, we must scale them in preparation for PCA
  * dimensionality reducer (i.e. PCA, SelectKBest) : reduce dimensionality of the data or selectively choose features
  * regressor: our prediction component

### 6.1 Model 1 - Standard Scaler, PCA, Linear Regression<a id='6.1_Model_1_-_StandardScaler_PCA_LinearRegression'></a>

Our first model uses a basic pipeline consisting of `StandardScaler`, `PCA`, and `LinearRegression`.

In [11]:
pipe_1 = make_pipeline(
    StandardScaler(), 
    PCA(random_state=42),
    LinearRegression() )
pipe_1.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'standardscaler', 'pca', 'linearregression', 'standardscaler__copy', 'standardscaler__with_mean', 'standardscaler__with_std', 'pca__copy', 'pca__iterated_power', 'pca__n_components', 'pca__random_state', 'pca__svd_solver', 'pca__tol', 'pca__whiten', 'linearregression__copy_X', 'linearregression__fit_intercept', 'linearregression__n_jobs', 'linearregression__normalize'])

In [12]:
param_grid_1 = {'pca__n_components': np.arange(1,20)}
model_1 = GridSearchCV(pipe_1, param_grid_1, cv=5)
model_1.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('pca', PCA(random_state=42)),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'pca__n_components': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])})

In [13]:
print("Model 1: Best Score: " + str(model_1.best_score_))
print("Model 1: Best Parameters: " + str(model_1.best_params_))

Model 1: Best Score: 0.23760770278422455
Model 1: Best Parameters: {'pca__n_components': 17}


### 6.2 Model 2 - Robust Scaler, PCA, Linear Regression<a id='6.2_Model_2_-_RobustScaler_PCA_LinearRegression'></a>

Our second model will try to improve on the first by switching out the `StandardScaler` for the `RobustScaler`.

In [14]:
pipe_2 = make_pipeline(
    RobustScaler(), 
    PCA(random_state=42),
    LinearRegression() )
pipe_2.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'robustscaler', 'pca', 'linearregression', 'robustscaler__copy', 'robustscaler__quantile_range', 'robustscaler__with_centering', 'robustscaler__with_scaling', 'pca__copy', 'pca__iterated_power', 'pca__n_components', 'pca__random_state', 'pca__svd_solver', 'pca__tol', 'pca__whiten', 'linearregression__copy_X', 'linearregression__fit_intercept', 'linearregression__n_jobs', 'linearregression__normalize'])

In [15]:
param_grid_2 = {'pca__n_components': np.arange(1,20)}
model_2 = GridSearchCV(pipe_2, param_grid_2, cv=5)
model_2.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('robustscaler', RobustScaler()),
                                       ('pca', PCA(random_state=42)),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'pca__n_components': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])})

In [16]:
print("Model 2: Best Score: " + str(model_2.best_score_))
print("Model 2: Best Parameters: " + str(model_2.best_params_))

Model 2: Best Score: 0.26785265231223887
Model 2: Best Parameters: {'pca__n_components': 17}


### 6.3 Model 3 - Robust Scaler, SelectKBest, Linear Regression<a id='6.3_Model_3_-_RobustScaler_SelectKBest_LinearRegression'></a>

In our third model, we'll swap out `PCA` for `SelectKBest`.

In [17]:
pipe_3 = make_pipeline(
    RobustScaler(), 
    SelectKBest(mutual_info_regression),
    LinearRegression() )
pipe_3.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'robustscaler', 'selectkbest', 'linearregression', 'robustscaler__copy', 'robustscaler__quantile_range', 'robustscaler__with_centering', 'robustscaler__with_scaling', 'selectkbest__k', 'selectkbest__score_func', 'linearregression__copy_X', 'linearregression__fit_intercept', 'linearregression__n_jobs', 'linearregression__normalize'])

In [18]:
param_grid_3 = {'selectkbest__k': np.arange(5,50) }
model_3 = GridSearchCV(pipe_3, param_grid_3, cv=5)
model_3.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('robustscaler', RobustScaler()),
                                       ('selectkbest',
                                        SelectKBest(score_func=<function mutual_info_regression at 0x000002A37E01DCA0>)),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'selectkbest__k': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])})

In [19]:
print("Model 3: Best Score: " + str(model_3.best_score_))
print("Model 3: Best Parameters: " + str(model_3.best_params_))

Model 3: Best Score: 0.32492821813385764
Model 3: Best Parameters: {'selectkbest__k': 45}


### 6.4 Model 4 - Robust Scaler, PCA, Random Forest<a id='6.4_Model_4_-_RobustScaler_PCA_RandomForest'></a>

In the above models, we seem to have determined that the best number of components for PCA is 17. In the fourth model, we will swap out the `LinearRegressor` for `RandomForestRegressor`.

In [20]:
pipe_4 = make_pipeline(
    RobustScaler(), 
    PCA(random_state=42, n_components=17),
    RandomForestRegressor(random_state=42) )
pipe_4.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'robustscaler', 'pca', 'randomforestregressor', 'robustscaler__copy', 'robustscaler__quantile_range', 'robustscaler__with_centering', 'robustscaler__with_scaling', 'pca__copy', 'pca__iterated_power', 'pca__n_components', 'pca__random_state', 'pca__svd_solver', 'pca__tol', 'pca__whiten', 'randomforestregressor__bootstrap', 'randomforestregressor__ccp_alpha', 'randomforestregressor__criterion', 'randomforestregressor__max_depth', 'randomforestregressor__max_features', 'randomforestregressor__max_leaf_nodes', 'randomforestregressor__max_samples', 'randomforestregressor__min_impurity_decrease', 'randomforestregressor__min_impurity_split', 'randomforestregressor__min_samples_leaf', 'randomforestregressor__min_samples_split', 'randomforestregressor__min_weight_fraction_leaf', 'randomforestregressor__n_estimators', 'randomforestregressor__n_jobs', 'randomforestregressor__oob_score', 'randomforestregressor__random_state', 'randomforestregressor__verbose

In [21]:
param_grid_4 = {'randomforestregressor__n_estimators': [int(n) for n in np.logspace(start=1, stop=3, num=5)],
                'randomforestregressor__max_depth': [None, 2, 5, 10, 20]}
model_4 = GridSearchCV(pipe_4, param_grid_4, cv=5, n_jobs=-1)
model_4.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('robustscaler', RobustScaler()),
                                       ('pca',
                                        PCA(n_components=17, random_state=42)),
                                       ('randomforestregressor',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'randomforestregressor__max_depth': [None, 2, 5, 10,
                                                              20],
                         'randomforestregressor__n_estimators': [10, 31, 100,
                                                                 316, 1000]})

In [22]:
print("Model 4: Best Score: " + str(model_4.best_score_))
print("Model 4: Best Parameters: " + str(model_4.best_params_))

Model 4: Best Score: 0.46721976012930444
Model 4: Best Parameters: {'randomforestregressor__max_depth': None, 'randomforestregressor__n_estimators': 1000}


## 7 Model Evaluation<a id='7_Model_Evaluation'></a>

Let's evaluate the models against each other.

In [37]:
models = [model_1, model_2, model_3, model_4]
print("          R2 score        Mean Absolute Error   Mean Squared Error")
for i, model in enumerate(models, start=1):
    y_pred = model.best_estimator_.predict(X_test)
    print("Model {}:  {:0.8f}      {:0,.8f}      {:0,.8f}".format(i, r2_score(y_test, y_pred), mean_absolute_error(y_test, y_pred), mean_squared_error(y_test, y_pred)))

          R2 score        Mean Absolute Error   Mean Squared Error
Model 1:  0.27343103      291,680.72941692      149,544,412,073.74673462
Model 2:  0.23920643      298,509.30007048      156,588,613,000.12109375
Model 3:  0.09225465      299,635.63912172      186,834,628,541.36068726
Model 4:  0.24424195      282,160.94451843      155,552,187,172.16000366


None of these models are performing very well...